In [ ]:
%run "{os.path.dirname(os.getcwd())}\general_functions\generalFunctions.ipynb" #container


## PPTX Functions

### Brand Elasticity Slide

In [ ]:
def BrandElasticity(prs, brandElasticity, numOfDuplication, cat, position=0):
    """
    Updates slides in a PowerPoint presentation with brand elasticity data.

    Parameters:
    prs (Presentation): The PowerPoint presentation object.
    brandElasticity (DataFrame): A DataFrame containing the brand elasticity data with columns 'Brand', 'PE down', and 'PE up'.
    numOfDuplication (int): Number of slides to update.
    position (int, optional): Starting slide position for the updates. Defaults to 0.
    """
    for slide_num in range(numOfDuplication):
        # Get the current slide
        slide = prs.slides[slide_num + position]
        shapes = slide.shapes
        
        # Create tables and charts (assuming createTableAndChart is a predefined function)
        tables, charts = createTableAndChart(shapes)
        
        y_axis_number = get_shape_number(shapes,"Priced up by +10%")
        shapes[y_axis_number].text = shapes[y_axis_number].text.replace('+10%', pricingPlus)
        shapes[y_axis_number].text_frame.paragraphs[0].font.size = Pt(8)
        shapes[y_axis_number].text_frame.paragraphs[0].font.color.rgb =RGBColor(87, 85, 85)
        shapes[y_axis_number].text_frame.paragraphs[0].font.name = 'Nexa Bold'
        shapes[y_axis_number].text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER

        x_axis_number = get_shape_number(shapes,"Priced down by -10%")
        shapes[x_axis_number].text = shapes[x_axis_number].text.replace('-10%', pricingMinus)
        shapes[x_axis_number].text_frame.paragraphs[0].font.size = Pt(8)
        shapes[x_axis_number].text_frame.paragraphs[0].font.color.rgb = RGBColor(87, 85, 85)
        shapes[x_axis_number].text_frame.paragraphs[0].font.name = 'Nexa Bold'
        shapes[x_axis_number].text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER

        collist= list(brandElasticity.columns)
        colname= collist[0]
        # Replace category or national information on the slide (assuming replaceCategoryOrNational is a predefined function)
        titleNumber = get_shape_number(shapes, "Based on price change | by Brand | Panda")  
        shapes[titleNumber].text = shapes[titleNumber].text.replace('Panda', marketsub)
        if colname!= "Brand":
            shapes[titleNumber].text = shapes[titleNumber].text.replace('by Brand', 'by Brand | '+colname)
        if colname== "Brand":
            shapes[titleNumber].text = shapes[titleNumber].text.replace('by Brand', 'by Brand | '+cat)
        shapes[titleNumber].text_frame.paragraphs[0].font.size = Pt(12)
        shapes[titleNumber - 1].text = data_source

            
        brandElasticity=brandElasticity.rename(columns={colname: "Brand",brandElasticity.columns[1]: "PE down",brandElasticity.columns[2]: "PE up"})
        # Get the first chart from the created charts
        chart = charts[0].chart
        
        # Create a new BubbleChartData object
        chart_data = BubbleChartData()
        chart_data.categories = ['']  # Add empty category for bubble chart
        
        # Add series to the chart data
        series = chart_data.add_series('PE up')
        
        # Add data points to the series
        for brand in brandElasticity["Brand"]:
            pe_down = brandElasticity[brandElasticity.Brand == brand]['PE down'].unique()[0]
            pe_up = brandElasticity[brandElasticity.Brand == brand]['PE up'].unique()[0]
            series.add_data_point(pe_down, pe_up, 1) 
            series.has_data_labels = True  # Enable data labels for the series
        
        # Replace chart data with the new data
        chart.replace_data(chart_data)
        
        category_axis = chart.category_axis
        value_axis = chart.value_axis

        # min and max for axes
        cat_min = float(round(pd.to_numeric(brandElasticity["PE down"],errors='coerce').min()-0.5)) # add margin
        cat_max = 0
        val_min = float(round(pd.to_numeric(brandElasticity["PE down"], errors='coerce').min()-0.5)) 
        val_max = 0

        category_axis.minimum_scale = cat_min
        category_axis.maximum_scale = 0
        value_axis.minimum_scale = val_min
        value_axis.maximum_scale = 0

        # Chart box boundaries
        chart_box = next((shp for shp in shapes if shp.name.lower() == "chartbox"), None)
        chart_left, chart_top = chart_box.left, chart_box.top
        chart_width, chart_height = chart_box.width, chart_box.height
        chart_right, chart_down = chart_left + chart_width, chart_top + chart_height

        # cross axis positions
        horiz_range = cat_max - cat_min
        vert_range  = val_max - val_min

        horiz_prop = (category_axis.maximum_scale - (-1)) / horiz_range
        vert_prop  = (value_axis.maximum_scale - (-1)) / vert_range

        horiz_pos = int(chart_left + horiz_prop * chart_width)
        vert_pos  = int(chart_top + (1 - vert_prop) * chart_height)

        # Quadrant colors
        quad_colors = {
            "low-low": RGBColor(255, 191, 191),
            "low-high": RGBColor(126, 202, 196),
            "high-low": RGBColor(126, 202, 196),
            "high-high": RGBColor(166, 218, 214),
        }

        for shp in shapes:
            if shp.name in quad_colors:
                shp.fill.solid()
                shp.fill.fore_color.rgb = quad_colors[shp.name]
                shp.fill.transparency = 0.6

                if shp.name == "low-high":
                    shp.left, shp.top = horiz_pos, chart_top
                    shp.width, shp.height = chart_right - horiz_pos, vert_pos - chart_top
                elif shp.name == "low-low":
                    shp.left, shp.top = chart_left, chart_top
                    shp.width, shp.height = horiz_pos - chart_left, vert_pos - chart_top
                elif shp.name == "high-high":
                    shp.left, shp.top = horiz_pos, vert_pos
                    shp.width, shp.height = chart_right - horiz_pos, chart_down - vert_pos
                elif shp.name == "high-low":
                    shp.left, shp.top = chart_left, vert_pos
                    shp.width, shp.height = horiz_pos - chart_left, chart_down - vert_pos

        # Update data labels for each point in the series
        for idx, point in enumerate(chart.series[0].points):
            data_label = point.data_label
            data_label.has_text_frame = True
            data_label.text_frame.text = brandElasticity.Brand[idx] 

### Brand Sourcing Slide

In [43]:
def BrandSourcingAnalysis(prs, plusbranding, Minusbranding, plusbrandingfair, Minusbrandingfair, numOfDuplication, position=0):
    """
    Updates slides in a PowerPoint presentation with brand sourcing analysis data.

    Parameters:
    prs (Presentation): The PowerPoint presentation object.
    plusbranding (DataFrame): DataFrame containing positive branding data.
    Minusbranding (DataFrame): DataFrame containing negative branding data.
    plusbrandingfair (DataFrame): DataFrame containing fair positive branding data.
    Minusbrandingfair (DataFrame): DataFrame containing fair negative branding data.
    numOfDuplication (int): Number of slides to update.
    position (int, optional): Starting slide position for the updates. Defaults to 0.
    """
    for slide_num in range(numOfDuplication):
        # Get the current slide
        slide = prs.slides[slide_num + position]
        col = plusbranding.columns[slide_num + 1]
        
        # Create tables and charts (assuming createTableAndChart is a predefined function)
        tables, charts = createTableAndChart(slide.shapes)

        # Replace category or national information on the slide
        shapes = prs.slides[slide_num + position].shapes
        titleNumber = get_shape_number(shapes, "Sourcing Analysis | Hershey's (2.8%) | Up & Down Pricing | Panda | Assumes no other are changing prices")
        #if titleNumber is not None:
        shapes[titleNumber - 1].text = data_source
        shapes[titleNumber].text = shapes[titleNumber].text.replace("Hershey's (2.8%)", col).replace('Panda', marketsub)
        shapes[titleNumber].text_frame.paragraphs[0].font.size = Pt(12)
        shapes[titleNumber].text_frame.paragraphs[0].font.name = 'Nexa Bold (Headings)'
        shapes[titleNumber + 1].text_frame.paragraphs[0].font.size = Pt(16)
        shapes[titleNumber + 1].text_frame.paragraphs[0].font.name = 'Nexa Bold (Headings)'


        
        # Merge and process the data for negative branding
        dfMinus = Minusbranding[['Product', col]].merge(Minusbrandingfair[['Product', col]], on='Product', suffixes=('', '_y'))
        dfMinus[col] = dfMinus[col] * (-1)
        dfMinus = dfMinus.sort_values(by=col, ascending=False).iloc[:10].reset_index(drop=True)
        dfMinus = dfMinus.fillna(0)
        dfMinus = dfMinus[dfMinus['Product'] != col]
        for column in dfMinus.columns:
            if column != 'Product':
                dfMinus = dfMinus[dfMinus[column] != -1]
        dfMinus = dfMinus.reset_index(drop=True)

        # Merge and process the data for positive branding
        dfplus = plusbranding[['Product', col]].merge(plusbrandingfair[['Product', col]], on='Product', suffixes=('', '_y'))
        dfplus = dfplus.sort_values(by=col, ascending=False).iloc[:10].reset_index(drop=True)
        dfplus = dfplus.fillna(0)
        dfplus = dfplus[dfplus['Product'] != col]
        for column in dfplus.columns:
            if column != 'Product':
                dfplus = dfplus[dfplus[column] != -1]
        dfplus = dfplus.reset_index(drop=True)

        
        # Get top 3 products based on fair data for positive and negative branding
        top3plus = dfplus.groupby("Product").apply(lambda x: x[col + "_y"].max()).nlargest(3).index.tolist()
        top3Minus = dfMinus.groupby("Product").apply(lambda x: x[col + "_y"].max()).nlargest(3).index.tolist()
        
        # Update the first chart with negative branding data
        chart_data = CategoryChartData()
        chart_data.categories = dfMinus['Product']
        chart_data.add_series(col, dfMinus[col])
        charts[0].chart.replace_data(chart_data)
        
        # Adjust the table size and remove extra rows
        table = tables[0].table
        table1 = tables[1].table
        num_rows_to_remove = len(tables[0].table.rows) - dfMinus.shape[0] - 1
        num_rows_to_remove1 = len(tables[1].table.rows) - dfplus.shape[0] - 1
        #table_height = 3.84
        table = removeRowFromTable(tables[0].table, num_rows_to_remove, rowToExclude=1)
        table1 = removeRowFromTable(tables[1].table, num_rows_to_remove1, rowToExclude=1)
        
        # Update the table cells with negative branding data
        for row_number, row in enumerate(table.rows, start=0):
            for column_num, cell in enumerate(row.cells):
                if row_number == 0 and column_num == 1:
                    cell.text = "Pricing (" + pricingMinus + ")"
                    set_cell_font(cell, 'Nexa Bold', 9)
                    cell.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER
                    cell.text_frame.paragraphs[0].font.bold = True
                if row_number > 0 and column_num == 2:
                    if dfMinus["Product"][row_number - 1] == col:
                        remove_row(table, row)
                    else:
                        value = dfMinus["Product"][row_number - 1]
                        cell.text = str(value)
                        set_cell_font(cell, 'Nexa Bold', 8)
                        cell.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER
                if row_number > 0 and column_num == 0:
                    value = dfMinus[col + '_y'][row_number - 1]
                    cell.text = str(round(value, 1))
                    set_cell_font(cell, 'Nexa Book', 8)
                    cell.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER
                    if dfMinus["Product"][row_number - 1] in top3Minus:
                        set_cell_font(cell, 'Nexa Bold', 8)
                        cell.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER
                        cell.text_frame.paragraphs[0].font.bold = True
        
        # Update the second chart with positive branding data
        chart_data1 = CategoryChartData()
        chart_data1.categories = dfplus['Product']
        chart_data1.add_series(col, dfplus[col])
        charts[1].chart.replace_data(chart_data1)
        
        # Update the table cells with positive branding data
        for row_number, row in enumerate(table1.rows, start=0):
            for column_num, cell in enumerate(row.cells):
                if row_number == 0 and column_num == 1:
                    cell.text = "Pricing (" + pricingPlus + ")"
                    set_cell_font(cell, 'Nexa Bold', 9)
                    cell.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER
                    cell.text_frame.paragraphs[0].font.bold = True
                if row_number > 0 and column_num == 0:
                    value = dfplus["Product"][row_number - 1]
                    cell.text = str(value)
                    set_cell_font(cell, 'Nexa Bold', 8)
                    cell.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER
                if row_number > 0 and column_num == 2:
                    if dfplus[col + '_y'][row_number - 1] == col:
                        remove_row(table1, row)
                    else:
                        value = dfplus[col + '_y'][row_number - 1]
                        cell.text = str(round(value, 1))
                        set_cell_font(cell, 'Nexa Book', 8)
                        cell.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER
                        if dfplus["Product"][row_number - 1] in top3plus:
                            set_cell_font(cell, 'Nexa Bold', 8)
                            cell.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER
                            cell.text_frame.paragraphs[0].font.bold = True


### Product Sourcing Slide

In [44]:
def ProductSourcingAnalysis(prs, plus, Minus, plusfair, Minusfair, numOfDuplication, position=0): 
    """
    Updates slides in a PowerPoint presentation with product sourcing analysis data.

    Parameters:
    prs (Presentation): The PowerPoint presentation object.
    plus (DataFrame): DataFrame containing positive product data.
    Minus (DataFrame): DataFrame containing negative product data.
    plusfair (DataFrame): DataFrame containing fair positive product data.
    Minusfair (DataFrame): DataFrame containing fair negative product data.
    numOfDuplication (int): Number of slides to update.
    position (int, optional): Starting slide position for the updates. Defaults to 0.
    """
    for slide_num in range(numOfDuplication):
        # Get the current slide
        slide = prs.slides[slide_num + position]
        
        # Determine the current column based on slide number
        col = plus.columns[slide_num + 1]
        
        # Create tables and charts (assuming createTableAndChart is a predefined function)
        tables, charts = createTableAndChart(slide.shapes)

        # Update the slide text and formatting and Replace category or national information on the slide
        shapes = prs.slides[slide_num + position].shapes
        titleNumber = get_shape_number(shapes, "Sourcing Analysis | Hershey's Choco Tubes Cookies N' Creme 18g (0.1%) | Up & Down Pricing | Panda | Assumes no other are changing prices")
        #if titleNumber is not None: Sourcing Analysis | Hershey's Choco Tubes Cookies N' Creme 18g (0.1%) | Up & Down Pricing | Panda | Assumes no other are changing prices

        shapes[titleNumber - 1].text = data_source
        shapes[titleNumber].text = shapes[titleNumber].text.replace("Hershey's Choco Tubes Cookies N' Creme 18g (0.1%)", col).replace('Panda', marketsub)
        shapes[titleNumber].text_frame.paragraphs[0].font.size = Pt(12)
        shapes[titleNumber].text_frame.paragraphs[0].font.name = 'Nexa Bold (Headings)'
        shapes[titleNumber + 1].text_frame.paragraphs[0].font.size = Pt(16)
        shapes[titleNumber + 1].text_frame.paragraphs[0].font.name = 'Nexa Bold (Headings)'

        # Merge and process the data for negative branding
        dfMinus = Minus[['Product', col]].merge(Minusfair[['Product', col]], on='Product', suffixes=('', '_y'))
        dfMinus[col] = dfMinus[col] * (-1)
        dfMinus = dfMinus.sort_values(by=col, ascending=False).iloc[:10].reset_index(drop=True)
        dfMinus = dfMinus.fillna(0)
        dfMinus = dfMinus[dfMinus['Product'] != col]
        
        # Merge and process the data for positive branding
        dfplus = plus[['Product', col]].merge(plusfair[['Product', col]], on='Product', suffixes=('', '_y'))
        dfplus = dfplus.sort_values(by=col, ascending=False).iloc[:10].reset_index(drop=True)
        dfplus = dfplus.fillna(0)
        dfplus = dfplus[dfplus['Product'] != col]
        
        # Get top 3 products based on fair data for positive and negative branding
        top3plus = dfplus.groupby("Product").apply(lambda x: x[col + "_y"].max()).nlargest(3).index.tolist()
        top3Minus = dfMinus.groupby("Product").apply(lambda x: x[col + "_y"].max()).nlargest(3).index.tolist()
        
        # Update the first chart with negative branding data
        chart_data = CategoryChartData()
        chart_data.categories = dfMinus['Product']
        chart_data.add_series(col, dfMinus[col])
        charts[0].chart.replace_data(chart_data)
        
        # Adjust the table size and remove extra rows
        table = tables[0].table
        table1 = tables[1].table
        num_rows_to_remove = len(tables[0].table.rows) - dfMinus.shape[0] - 1
        num_rows_to_remove1 = len(tables[1].table.rows) - dfMinus.shape[0] - 1
        #table_height = 3.84
        table = removeRowFromTable(tables[0].table, num_rows_to_remove, rowToExclude=1)
        table1 = removeRowFromTable(tables[1].table, num_rows_to_remove1, rowToExclude=1)

        # Update the table cells with negative branding data
        for row_number, row in enumerate(tables[0].table.rows, start=0):
            for column_num, cell in enumerate(row.cells):
                if row_number == 0 and column_num == 1:
                    cell.text = "Pricing (" + pricingMinus + ")"
                    set_cell_font(cell, 'Nexa Bold', 9)
                    cell.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER
                    cell.text_frame.paragraphs[0].font.bold = True
                    
                if row_number > 0 and column_num == 2:
                    if dfMinus["Product"][row_number - 1] == col:
                        remove_row(tables[0].table, row)
                    else:
                        value = dfMinus["Product"][row_number - 1]
                        cell.text = str(value)
                        set_cell_font(cell, 'Nexa Bold', 8)
                        cell.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER
                    
                if row_number > 0 and column_num == 0:
                    value = dfMinus[col + '_y'][row_number - 1]
                    cell.text = str(round(value, 1))
                    set_cell_font(cell, 'Nexa Book', 8)
                    cell.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER
                    if dfMinus["Product"][row_number - 1] in top3Minus:
                        set_cell_font(cell, 'Nexa Bold', 8)
                        cell.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER
                        cell.text_frame.paragraphs[0].font.bold = True
                    
        # Update the second chart with positive branding data
        chart_data1 = CategoryChartData()
        chart_data1.categories = dfplus['Product']
        chart_data1.add_series(col, dfplus[col])
        charts[1].chart.replace_data(chart_data1)
        
        # Update the table cells with positive branding data
        for row_number, row in enumerate(tables[1].table.rows, start=0):
            for column_num, cell in enumerate(row.cells):
                if row_number == 0 and column_num == 1:
                    cell.text = "Pricing (" + pricingPlus + ")"
                    set_cell_font(cell, 'Nexa Bold', 9)
                    cell.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER
                    cell.text_frame.paragraphs[0].font.bold = True
                    
                if row_number > 0 and column_num == 0:
                    value = dfplus["Product"][row_number - 1]
                    cell.text = str(value)
                    set_cell_font(cell, 'Nexa Bold', 8)
                    cell.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER
                    
                if row_number > 0 and column_num == 2:
                    if dfplus[col + '_y'][row_number - 1] == col:
                        remove(tables[1].table, row)
                    else:
                        value = dfplus[col + '_y'][row_number - 1]
                        cell.text = str(round(value, 1))
                        set_cell_font(cell, 'Nexa Book', 8)
                        cell.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER
                        if dfplus["Product"][row_number - 1] in top3plus:
                            set_cell_font(cell, 'Nexa Bold', 8)
                            cell.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER
                            cell.text_frame.paragraphs[0].font.bold = True


### PE Slide

In [ ]:
def PriceElasticityCurve(prs, merged, group_list, duplication, position=0):
    """
    Updates slides in a PowerPoint presentation with price elasticity curve data.
 
    Parameters:
    prs (Presentation): The PowerPoint presentation object.
    merged (DataFrame): DataFrame containing merged data for price elasticity analysis.
    group_list (list): List of group names corresponding to each slide.
    duplication (int): Number of slides to update.
    position (int, optional): Starting slide position for the updates. Defaults to 0.
    """
    for group, slide_num in zip(group_list, range(duplication)):
        # Get the current slide
        slide = prs.slides[slide_num + position]
       
        # Update the slide text and formatting and Replace category or national information on the slide
        shapes = prs.slides[slide_num + position].shapes
        titleNumber = get_shape_number(shapes, "Hershey's | Price Elasticity | Panda | Assumes no other products are changing prices")
        #if titleNumber is not None:
        shapes[titleNumber - 1].text = data_source
        shapes[titleNumber].text = shapes[titleNumber].text.replace("Hershey's", group).replace('Panda', marketsub)
        shapes[titleNumber].text_frame.paragraphs[0].font.size = Pt(12)
        shapes[titleNumber].text_frame.paragraphs[0].font.name = 'Nexa Bold (Headings)'
        shapes[titleNumber + 1].text_frame.paragraphs[0].font.size = Pt(16)
        shapes[titleNumber + 1].text_frame.paragraphs[0].font.name = 'Nexa Bold (Headings)'
 
       
        # Filter data for the current group
        data = merged[merged['Grouping'] == group]
        data['GM%'].replace(np.nan, '', inplace=True)
        data['WOB%'].replace(np.nan, '', inplace=True)
        data.fillna(0, inplace=True)
        data = data.reset_index(drop=True)
 
        # Create tables and charts (assuming createTableAndChart is a predefined function)
        tables, charts = createTableAndChart(slide.shapes)
 
        # Initialize XY chart data
        chart_data = XyChartData()
        chart_data.categories = data['Product']
        axis = []
 
        # Loop through each product and add data points to the chart
        for name, num in zip(data['Product'], range(len(data['Product']))):
            series = chart_data.add_series(name)
            for i in range(5):
                column_name = f'P{i+1}'
                series.add_data_point(data[column_name][num], data[f"Share {column_name} - Base Share"][num])
                charts[0].chart.replace_data(chart_data)
                pe_value = data[f"PE {column_name}-P{i+2}"][num]
                if column_name == 'P3':
                    label_text = '-'
                else:
                    label_text = str(round(pe_value,1)) if pe_value != 0 else "-"
                charts[0].chart.series[num].points[i].data_label.has_text_frame = True
                charts[0].chart.series[num].points[i].data_label.text_frame.text = label_text
               
                # Add color to points
                charts[0].chart.series[num].marker.format.fill.solid()
                charts[0].chart.series[num].marker.format.fill.fore_color.rgb = colorList[num]
                charts[0].chart.series[num].format.line.color.rgb = colorList[num]
               
                axis.append(data[column_name][num])
 
        # Set chart axis scale
        charts[0].chart.category_axis.minimum_scale = float(min(axis))
        charts[0].chart.category_axis.maximum_scale = float(max(axis))
        if decimals == 2:
            charts[0].chart.category_axis.tick_labels.number_format = '#,##0.00'
        elif decimals==0:
            charts[0].chart.category_axis.tick_labels.number_format = '#,##0'
        else:
            charts[0].chart.category_axis.tick_labels.number_format = '#,##0.00'
       
        # Update chart title

        charts[0].chart.replace_data(chart_data)
       
        category_axis = charts[0].chart.category_axis
        currencywithoutspace =currency.strip()  # Remove the leading space
        category_axis.axis_title.text_frame.text = f"Tested Prices ({currencywithoutspace})"
        category_axis.axis_title.text_frame.paragraphs[0].font.name = "Nexa Bold"
        category_axis.axis_title.text_frame.paragraphs[0].font.size = Pt(8)
        category_axis.axis_title.text_frame.paragraphs[0].font.color.rgb = RGBColor(87, 85, 85)
 
        # Create a new DataFrame for the table
        new = data[['Product', 'PE Down', 'PE Up', 'WOB%', 'GM%']]
        table = tables[0].table
 
        # Remove extra rows from the table
        num_rows_to_remove = len(table.rows) - new.shape[0] - 1
        for _ in range(num_rows_to_remove):
            if len(table.rows) > 1:  # Skip removing the first row if there is more than one row
                row = table.rows[1]
                remove_row(table, row)
 
        # Update the table cells with new data
        for row_number, row in enumerate(tables[0].table.rows, start=0):
            for col, cell in enumerate(row.cells):
                if row_number == 0:
                    continue  # Skip the header row
               
                if col == 0:
                    fill = cell.fill
                    fill.solid()
                    fill.fore_color.rgb = colorList[row_number - 1]
                   
                elif col == 1:
                    value = new.iloc[row_number - 1, col - 1]
                    cell.text = str(value)
                    set_cell_font(cell, 'Nexa Book', 8)
                    cell.text_frame.paragraphs[0].alignment = PP_ALIGN.LEFT
               
                elif col == 2:
                    value = new.iloc[row_number - 1, col - 1]
                    if value <= -1.0000001:
                        cell.text = str(round(value, 1))
                        set_cell_font(cell, 'Nexa Book', 8)
                        cell.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER
                        fill = cell.fill
                        fill.solid()
                        fill.fore_color.rgb = RGBColor(126, 202, 196)
               
                    else:
                        cell.text = str(round(value, 1))
                        set_cell_font(cell, 'Nexa Book', 8)
                        cell.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER
                        fill = cell.fill
                        fill.solid()
                        fill.fore_color.rgb = RGBColor(255, 255, 255)
               
                   
                   
                elif col == 3:
                    value = new.iloc[row_number - 1, col - 1]
                    if value >=-1:
                        cell.text = str(round(value, 1))
                        set_cell_font(cell, 'Nexa Book', 8)
                        cell.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER
                        fill = cell.fill
                        fill.solid()
                        fill.fore_color.rgb = RGBColor(126, 202, 196)
               
                    else:
                        cell.text = str(round(value, 1))
                        set_cell_font(cell, 'Nexa Book', 8)
                        cell.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER
                        fill = cell.fill
                        fill.solid()
                        fill.fore_color.rgb = RGBColor(255, 255, 255)
               
                elif col == 4:
                    value = new.iloc[row_number - 1, col - 1]
                    value=value * 100
                    try:
                        valuepct=str(round(value, 1))+ "%"
                    except:
                        valuepct=''
                    cell.text = str(valuepct)
                    set_cell_font(cell, 'Nexa Book', 8)
                    cell.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER
 
                elif col ==5:
                    value = new.iloc[row_number - 1, col - 1]
                    value=value * 100
                    try:
                        valuepct=str(round(value, 1))+ "%"
                    except:
                        valuepct=''
                    cell.text = str(valuepct)
                    set_cell_font(cell, 'Nexa Book', 8)
                    cell.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER
               
                else:
                    cell.text = ""

### Revenue Response Slide

In [ ]:
def update_value_axis(chart, min_val, max_val):
    """
    Updates the value axis with specified minimum and maximum values.
 
    Parameters:
    chart : Chart object
        The chart object whose value axis needs to be updated.
    min_val : float
        The minimum value for the value axis scale.
    max_val : float
        The maximum value for the value axis scale.
    """
    # Access the part and XML element of the chart
    chart_element = chart.element
 
    # Define the XML namespaces
    namespaces = {
        'c': 'http://schemas.openxmlformats.org/drawingml/2006/chart',
    }
 
    # Find the value axis in the chart XML
    value_axis = chart_element.find('.//c:valAx', namespaces)
   
    if value_axis is None:
        raise ValueError("Value axis not found in the chart.")
 
    # Ensure <c:scaling> element exists
    scaling = value_axis.find('.//c:scaling', namespaces)
    if scaling is None:
        scaling = OxmlElement('c:scaling')
        value_axis.append(scaling)
 
    # Update minimum and maximum values
    min_scale = scaling.find('.//c:min', namespaces)
    if min_scale is None:
        min_scale = OxmlElement('c:min')
        scaling.append(min_scale)
    min_scale.set('val', str(min_val))
   
    max_scale = scaling.find('.//c:max', namespaces)
    if max_scale is None:
        max_scale = OxmlElement('c:max')
        scaling.append(max_scale)
    max_scale.set('val', str(max_val))
 
    print(f"Updated value axis to min: {min_val}, max: {max_val}")
 
 
 

In [ ]:
def revenue_response(prs, dfList, numOfDuplication, position=0):
    """
    Updates slides in a PowerPoint presentation with revenue response data.

    Parameters:
    prs (Presentation): The PowerPoint presentation object.
    dfList (list): List of DataFrames containing revenue response data for each slide.
    numOfDuplication (int): Number of slides to update.
    position (int, optional): Starting slide position for the updates. Defaults to 0.
    """
    seri = {
        'Volume Ix': 'Share Index',
        'Value Ix': 'Rev Index',
        'Gross Profit Ix': 'Profit',
        'Gross Margin %': 'Margin'
    }
    
    for df, slide_num in zip(dfList, range(numOfDuplication)):
        # Filter columns related to pricing
        df = df.reset_index(drop=True)
        #cols = [col for col in df.columns if ('-'+currency.lower() in col[:4].lower()) or ('+'+currency.lower() in col[:4].lower()) or ('Dashboard price' == col)]
        cols = [col for col in df.columns if ('-' in col[:4].lower()) or ('+' in col[:4].lower()) or ('Dashboard price' == col)]
        final = df[['Product'] + [col for col in df.columns if any(col.endswith(j) for j in cols)]]
        # Generate list of price categories
        cat_list = (final[[col for col in final.columns if col.startswith(('-', '+', 'Dashboard'))]].iloc[0]).tolist()

        cat_list = [format_number(float(col), use_decimals=True, decimals=decimals, currency_symbol=None, currency_before= False if sign.lower() == 'after' else True).replace(' ','') for col in cat_list]
        #cat_list = [currency+' {:.2f}'.format(float(col)) for col in cat_list]
    
        # Get the current slide
        slide = prs.slides[slide_num + position]

        # Update slide text and formatting
        shapes = prs.slides[slide_num + position].shapes
        titleNumber = get_shape_number(shapes, "Revenue Response Curve by Price Point | Hershey's Choco Tubes Cookies N' Creme 18g | Panda | Assuming no other products are changing prices")
        shapes[titleNumber - 1].text = data_source
        shapes[titleNumber].text = shapes[titleNumber].text.replace("Hershey's Choco Tubes Cookies N' Creme 18g ",df['Product'][0] + " ").replace('Panda', marketsub)
        shapes[titleNumber].text_frame.paragraphs[0].font.size = Pt(12)
        shapes[titleNumber].text_frame.paragraphs[0].font.name = 'Nexa Bold (Headings)'
        shapes[titleNumber + 1].text_frame.paragraphs[0].font.size = Pt(16)
        shapes[titleNumber + 1].text_frame.paragraphs[0].font.name = 'Nexa Bold (Headings)'

        
        # Create tables and charts (assuming createTableAndChart is a predefined function)
        tables, charts = createTableAndChart(slide.shapes)

        # Initialize chart data
        chart_data = CategoryChartData()
        chart_data.categories = cat_list
        
        # Add series to the chart data
        # for key, value in seri.items():
        #     chart_data.add_series(key, (final[[col for col in final.columns if value in col]].iloc[0]).tolist())
        # Assuming 'final' is your DataFrame and 'seri' is your dictionary

        for key, value in seri.items():
            # Extract the columns from 'final' that contain 'value' in their name
            columns = [col for col in final.columns if value in col]
            
            # Get the first row of these columns and convert it to a list of numbers
            series_data = final[columns].iloc[0].astype(float).tolist()
            
            # Add the series data to 'chart_data'
            chart_data.add_series(key, series_data)

        # Update the chart with new data
        charts[0].chart.replace_data(chart_data)


        value_axis = charts[0].chart.value_axis
        
        category_axis = charts[0].chart.category_axis

        currencywithoutspace =currency.strip()  # Remove the leading space
        category_axis.axis_title.text_frame.text = f"Shelf Price/Unit ({currencywithoutspace})"
        category_axis.axis_title.text_frame.paragraphs[0].font.name = "Nexa Bold"
        category_axis.axis_title.text_frame.paragraphs[0].font.size = Pt(8)
        category_axis.axis_title.text_frame.paragraphs[0].font.color.rgb = RGBColor(87, 85, 85)
        # Get the minimum and maximum values for the secondary axis
        gm_values = []
        for key, value in seri.items():
            columns = [col for col in final.columns if "Gross Margin" in col]
            series_data = final[columns].iloc[0].astype(float).tolist()
            gm_values.extend(series_data)  # Combine all the y-values into one list for min/max calculation

        min_gm_value = round((1-(0.2*np.sign(min(gm_values))))*min(gm_values),1)
        max_gm_value =round((1+(0.2*np.sign(max(gm_values))))*max(gm_values),1)
        value_axis.minimum_scale = min_gm_value
        value_axis.maximum_scale = max_gm_value

        # Get the minimum and maximum values for the primary Y axis
        y_values = []
        for key, value in seri.items():
            columnsy = [col for col in final.columns if ("Gross Margin" not in col) and (value in col)]
            series_data = final[columnsy].iloc[0].astype(float).tolist()
            y_values.extend(series_data)  # Combine all the y-values into one list for min/max calculation

        if min(y_values)<-1000 or max(y_values)>1000:
            min_y_value = round(((1-(0.2*np.sign(min(y_values))))*min(y_values))/100,0)*100
            max_y_value =round(((1+(0.2*np.sign(max(y_values))))*max(y_values))/100,0)*100
        else:
            min_y_value = round(((1-(0.2*np.sign(min(y_values))))*min(y_values))/10,0)*10
            max_y_value =round(((1+(0.2*np.sign(max(y_values))))*max(y_values))/10,0)*10
        update_value_axis(charts[0].chart,min_y_value,max_y_value)

        

        # Get the x-axis scale (in case you want to adjust based on the number of series or data points)
        # In this case, we're using a simple range based on the data points
        #x_values = list(range(len(series_data)))  # Create a range based on the number of data points

        #min_x_value = min(x_values)
        #max_x_value = max(x_values)
        #category_axis.minimum_scale = 0.8 * min_x_value
        #category_axis.maximum_scale = 1.2 * max_x_value

In [ ]:
def AddRectangle(sl,left,width,color_text):
    color=color_text.split("|")[0]
    text=color_text.split("|")[1]
    color_map={"Yellow":RGBColor(243, 241, 80),"Green":RGBColor(7, 160, 152),"Red":RGBColor(255, 191, 191)}
    rec=sl.shapes.add_shape(MSO_SHAPE.RECTANGLE, left, Cm(4.05), width, Cm(7.78))
    rec.fill.solid()
    rec.fill.fore_color.rgb = color_map[color]
    def SubElement(parent, tagname, **kwargs):
            element = OxmlElement(tagname)
            element.attrib.update(kwargs)
            parent.append(element)
            return element

    def _set_shape_transparency(shape, alpha):
        """ Set the transparency (alpha) of a shape"""
        ts = shape.fill._xPr.solidFill
        sF = ts.get_or_change_to_srgbClr()
        sE = SubElement(sF, 'a:alpha', val=str(alpha))


    # rec.fill.transparency=0.5
    transparency={"Red":75,"Green":75,"Yellow":75}
    _set_shape_transparency(rec,(100-transparency[color])*1000)
    rec.text=text
    rec.text_frame.paragraphs[0].font.name = 'Nexa bold'  
    rec.text_frame.paragraphs[0].font.size = Pt(8)
    font_color={"Yellow":RGBColor(92, 91, 6),"Green":RGBColor(0, 160, 151),"Red":RGBColor(192, 0, 0)}
    rec.text_frame.paragraphs[0].font.color.rgb = font_color[color]  
    rec.text_frame.paragraphs[0].font.bold = True
    rec.text_frame.vertical_anchor = MSO_ANCHOR.TOP
    rec.line.fill.background()
    rec.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER
    return rec

In [ ]:
def DetermineShapeNumber(x):
    current=x[0]
    res=[]
    count=1
    for i in x :
        if i==current:
            res.append(count)
        else :
            count+=1
            current=i
            res.append(count)
    return res

### Size Elasticity

In [ ]:
def SE_Slide(prs, data, duplication, position):
    # Step 1: Split data into chunks of 20 rows
    chunk_size = 20
    data_chunks = [data.iloc[i:i + chunk_size] for i in range(0, len(data), chunk_size)]
    for slide_num, data_chunk in enumerate(data_chunks):
        print(f"Processing Slide {slide_num + 1}")
        slide = prs.slides[slide_num + position]
        shapes = slide.shapes        
        sourceNumber = get_shape_number(shapes, "DATA SOURCE: Consumer Test | May 2024")
        shapes[sourceNumber].text = data_source
        subtitleNumber = get_shape_number(shapes, "Size Elasticity | Traditional Trade | Assumes no other products are changing prices or sizes")
        shapes[subtitleNumber].text = shapes[subtitleNumber].text.replace('Traditional Trade', marketsub)
        shapes[subtitleNumber].text_frame.paragraphs[0].font.size = Pt(12)
        shapes[subtitleNumber].text_frame.paragraphs[0].font.name = 'Nexa Bold (Headings)'
               
        tables, charts = createTableAndChart(shapes)
        num_rows_to_remove = len(tables[0].table.rows) - data_chunk.shape[0] - 2
        table = removeRowFromTable(tables[0].table, num_rows_to_remove, rowToExclude=2)
               
        for row_number, row in enumerate(table.rows, start=0):
            for column_num, cell in enumerate(row.cells):
                if row_number > 1 and row_number - 2 < len(data_chunk):  # Adjust to chunk
                    index = row_number - 2
                    if column_num == 0:
                        cell.text = data_chunk['Product'].iloc[index]
                        set_cell_font(cell, 'Nexa Bold', 6)
                        cell.text_frame.paragraphs[0].alignment = PP_ALIGN.LEFT
                        cell.text_frame.paragraphs[0].font.bold = True
                    elif column_num == 1:
                        cell.text = str(data_chunk['Base Size'].iloc[index])
                        set_cell_font(cell, 'Nexa Book (Body)', 7)
                        cell.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER
                    elif column_num == 2:
                        cell.text = str(data_chunk['New Size'].iloc[index])
                        set_cell_font(cell, 'Nexa Book (Body)', 7)
                        cell.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER
                    elif column_num == 3:
                        cell.text = str(int(data_chunk['Size Change'].iloc[index] * 100)) + '%'
                        set_cell_font(cell, 'Nexa Book (Body)', 7)
                        cell.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER
                    elif column_num == 4:
                        cell.text = str(round(data_chunk['Volume Index'].iloc[index], 1))
                        set_cell_font(cell, 'Nexa Book (Body)', 7)
                        cell.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER
                    elif column_num == 5:
                        cell.text = str(round(data_chunk['Value Index'].iloc[index], 1))
                        set_cell_font(cell, 'Nexa Book (Body)', 7)
                        cell.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER
                    elif column_num == 6:
                        cell.text = str(round(data_chunk['Gross Profit'].iloc[index], 1))
                        set_cell_font(cell, 'Nexa Book (Body)', 7)
                        cell.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER
                    elif column_num == 7:
                        cell.text = str(round(data_chunk['Size Elasticity'].iloc[index], 1))
                        set_cell_font(cell, 'Nexa Book (Body)', 7)
                        cell.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER
                    elif column_num == 8:
                        cell.text = str(round(data_chunk['SCD PE'].iloc[index], 1))
                        set_cell_font(cell, 'Nexa Book (Body)', 7)
                        cell.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER
                    elif column_num == 9:
                        cell.text = str(round(data_chunk['PCD PE'].iloc[index], 1))
                        set_cell_font(cell, 'Nexa Book (Body)', 7)
                        cell.text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER